In [2]:
### ETAPES AU PROJET : 

#1. Définir des distances entre les bulles en fonction de la similarité des infos 
#2. Relier les bulles connectées entre elles 

import dash
from dash import html, dcc, Input, Output
import plotly.graph_objects as go
import networkx as nx
import pandas as pd



### Etude du fichier csv et compte des étiquettes 

df = pd.read_csv("Data_20250506.csv", sep=';')

etiquettes_data = df.iloc[:, 11:]  # à partir de la colonne 12, on prend tous les mots clés des études scientifiques 
etiquettes_series = pd.Series(etiquettes_data.values.ravel())

# Nettoyage des valeurs : on enlève les cases vides, les majuscules...car sinon on compterait différemment Fer et fer par exemple

etiquettes_series = etiquettes_series.dropna().astype(str).str.strip().str.lower() 

etiquettes_comptes = etiquettes_series.value_counts().reset_index()  # compte le nombre d'occurences dans toute la littérature scientifique
etiquettes_comptes.columns = ['Étiquette', 'Nombre_de_récurrence']

print(etiquettes_comptes)






UnicodeDecodeError: 'utf-8' codec can't decode byte 0x96 in position 25475: invalid start byte

In [ ]:



# Données
products = [
    {"id": "Fer", "group": "métaux", "value": 100},
    {"id": "Or", "group": "métaux", "value": 70},
    {"id": "cuivre", "group": "métaux", "value": 80},
    {"id": "T-Shirts", "group": "Textiles", "value": 20},
    {"id": "chaussures", "group": "Textiles", "value": 40}
]

edges = [("Fer", "Or"), ("Fer", "cuivre"), ("T-Shirts", "chaussures")]

# Création du graphe
G = nx.Graph()
for product in products:
    G.add_node(product["id"], group=product["group"], value=product["value"])
G.add_edges_from(edges)

pos = nx.spring_layout(G, seed=42)

# Mapping couleurs
color_map = {
    "métaux": "purple",
    "Textiles": "red"
}

def create_figure(highlight_node=None):
    # Noeuds
    node_x = []
    node_y = []
    node_text = []
    node_color = []
    node_size = []

    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_text.append(node)
        group = G.nodes[node]['group']
        node_color.append(color_map.get(group, 'grey'))
        node_size.append(G.nodes[node]['value'])

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        text=node_text,
        hoverinfo='text',
        marker=dict(
            showscale=False,
            color=node_color,
            size=node_size,
            line_width=2
        )
    )

    

    # Arêtes 
    edge_x = []
    edge_y = []
    if highlight_node:
        for edge in G.edges():
            if highlight_node in edge:
                x0, y0 = pos[edge[0]]
                x1, y1 = pos[edge[1]]
                edge_x += [x0, x1, None]
                edge_y += [y0, y1, None]

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=1, color='#888'),
        hoverinfo='none',
        mode='lines'
    )

    fig = go.Figure(data=[edge_trace, node_trace],
        layout=go.Layout(
            showlegend=False,
            hovermode='closest',
            margin=dict(b=20, l=5, r=5, t=40),
            xaxis=dict(showgrid=False, zeroline=False, visible=False),
            yaxis=dict(showgrid=False, zeroline=False, visible=False)
        )
    )
    return fig

    


# Dash App
app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1("Product space"),
    dcc.Graph(id='product-space', figure=create_figure())
])

@app.callback(
    Output('product-space', 'figure'),
    Input('product-space', 'hoverData')
)
def update_edges_on_hover(hoverData):
    if hoverData:
        node = hoverData['points'][0]['text']
        return create_figure(highlight_node=node)
    return create_figure()

if __name__ == '__main__':
    app.run(debug=True)
